# RBPF Volatility Filter Analysis

This notebook:
1. Fetches market data from yFinance
2. Runs RBPF filter
3. Visualizes volatility estimates and regime detection
4. Compares to realized volatility
5. Exports results to CSV

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Local modules
from rbpf import RBPF, ParamMode, AdaptSignal
from data import (
    fetch_returns, fetch_ohlcv, add_features,
    compute_realized_vol, compute_realized_vol_exp,
    save_rbpf_results, detect_outliers
)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

## 1. Configuration

In [ ]:
# === USER CONFIGURATION ===

TICKER = "SPY"          # Yahoo Finance ticker
PERIOD = "2y"           # Data period: 1y, 2y, 5y, max
INTERVAL = "1d"         # Data interval: 1d, 1h, etc.

# RBPF settings
N_PARTICLES = 512
N_REGIMES = 4

# Regime parameters (tune these for your asset)
# Format: (theta, mu_vol, sigma_vol)
# theta: mean reversion speed (higher = faster revert)
# mu_vol: long-run log-volatility (lower = calmer)
# sigma_vol: vol-of-vol (higher = more volatile volatility)

REGIME_PARAMS = {
    0: (0.003, -4.50, 0.08),   # Calm
    1: (0.042, -3.67, 0.267),  # Mild
    2: (0.081, -2.83, 0.453),  # Trend
    3: (0.120, -2.00, 0.640),  # Crisis
}

# Transition matrix (stickiness on diagonal)
STICKINESS = 0.92
TRANSITION_MATRIX = np.array([
    [0.920, 0.056, 0.020, 0.004],
    [0.032, 0.920, 0.036, 0.012],
    [0.012, 0.036, 0.920, 0.032],
    [0.004, 0.020, 0.056, 0.920]
], dtype=np.float32)

# OCSN outlier params (prob, variance)
OCSN_PARAMS = {
    0: (0.02, 100.0),
    1: (0.03, 120.0),
    2: (0.04, 140.0),
    3: (0.05, 160.0),
}

# Output
SAVE_CSV = True
OUTPUT_DIR = "./"

## 2. Fetch Data

In [ ]:
# Fetch OHLCV
df = fetch_ohlcv(TICKER, period=PERIOD, interval=INTERVAL)
df = add_features(df)

# Get returns
returns, dates = fetch_returns(TICKER, period=PERIOD, interval=INTERVAL)

print(f"Ticker: {TICKER}")
print(f"Date range: {dates[0].date()} to {dates[-1].date()}")
print(f"Observations: {len(returns)}")
print(f"\nReturn statistics:")
print(f"  Mean (daily): {returns.mean():.6f}")
print(f"  Std (daily):  {returns.std():.6f}")
print(f"  Std (annual): {returns.std() * np.sqrt(252):.2%}")
print(f"  Min: {returns.min():.4f}")
print(f"  Max: {returns.max():.4f}")

# Outliers
outliers = detect_outliers(returns, sigma=3.0)
print(f"\nOutliers (>3σ): {outliers.sum()}")

In [ ]:
# Plot price and returns
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Price
ax = axes[0]
ax.plot(df.index, df['Close'], 'b-', linewidth=1)
ax.set_ylabel('Price')
ax.set_title(f'{TICKER} Price')

# Returns
ax = axes[1]
ax.bar(dates, returns, width=1, color='steelblue', alpha=0.7)
ax.axhline(0, color='black', linewidth=0.5)
ax.set_ylabel('Return')
ax.set_title('Daily Returns')

# Highlight outliers
if outliers.any():
    ax.scatter(dates[outliers], returns[outliers], color='red', s=30, zorder=5, label='Outliers')
    ax.legend()

plt.tight_layout()
plt.show()

## 3. Run RBPF Filter

In [ ]:
# Create RBPF
rbpf = RBPF(n_particles=N_PARTICLES, n_regimes=N_REGIMES, mode=ParamMode.STORVIK)

# Configure regimes
for regime, (theta, mu_vol, sigma_vol) in REGIME_PARAMS.items():
    rbpf.set_regime_params(regime, theta, mu_vol, sigma_vol)

# Transition matrix
rbpf.set_transition_matrix(TRANSITION_MATRIX)

# Adaptive forgetting (regime mode = use baseline λ per regime)
rbpf.enable_adaptive_forgetting(AdaptSignal.REGIME)

# Circuit breaker
rbpf.enable_circuit_breaker(0.999, 100)

# Robust OCSN
rbpf.enable_ocsn()
for regime, (prob, variance) in OCSN_PARAMS.items():
    rbpf.set_ocsn_params(regime, prob, variance)

# Initialize (start in calm regime)
mu0 = REGIME_PARAMS[0][1]  # mu_vol of regime 0
rbpf.init(mu0=mu0, var0=0.1)

print(f"RBPF initialized: {N_PARTICLES} particles, {N_REGIMES} regimes")

In [ ]:
# Run filter
results = []

for i, ret in enumerate(returns):
    rbpf.step(ret)
    
    # Collect output
    results.append({
        'tick': i,
        'date': dates[i],
        'return': ret,
        'vol_mean': rbpf.vol_mean,
        'log_vol_mean': rbpf.log_vol_mean,
        'log_vol_var': rbpf.log_vol_var,
        'dominant_regime': rbpf.dominant_regime,
        'regime_prob_0': rbpf.regime_probs[0],
        'regime_prob_1': rbpf.regime_probs[1],
        'regime_prob_2': rbpf.regime_probs[2],
        'regime_prob_3': rbpf.regime_probs[3],
        'ess': rbpf.ess,
        'surprise': rbpf.surprise,
        'outlier_fraction': rbpf.outlier_fraction,
        'resampled': rbpf.resampled,
        'learned_mu_0': rbpf.learned_mu_vol[0],
        'learned_mu_1': rbpf.learned_mu_vol[1],
        'learned_mu_2': rbpf.learned_mu_vol[2],
        'learned_mu_3': rbpf.learned_mu_vol[3],
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.set_index('date', inplace=True)

print(f"Processed {len(results)} ticks")
print(f"\nRegime distribution:")
print(results_df['dominant_regime'].value_counts().sort_index())

## 4. Compute Realized Volatility for Comparison

In [ ]:
# Rolling realized vol (not annualized, for comparison with RBPF)
rv_20 = compute_realized_vol(returns, window=20, annualize=False)
rv_60 = compute_realized_vol(returns, window=60, annualize=False)
rv_exp = compute_realized_vol_exp(returns, halflife=20, annualize=False)

# Add to results
results_df['realized_vol_20'] = rv_20
results_df['realized_vol_60'] = rv_60
results_df['realized_vol_exp'] = rv_exp

## 5. Visualization

In [ ]:
# Color map for regimes
REGIME_COLORS = ['green', 'yellow', 'orange', 'red']
REGIME_NAMES = ['Calm', 'Mild', 'Trend', 'Crisis']

In [ ]:
# Main plot: Price, Volatility, Regime
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# 1. Price with regime background
ax = axes[0]
ax.plot(df.index[1:], df['Close'].values[1:], 'k-', linewidth=0.8)

# Shade by regime
for i in range(len(results_df)):
    regime = results_df['dominant_regime'].iloc[i]
    ax.axvspan(results_df.index[i], 
               results_df.index[min(i+1, len(results_df)-1)],
               alpha=0.3, color=REGIME_COLORS[regime], linewidth=0)

ax.set_ylabel('Price')
ax.set_title(f'{TICKER} Price with Regime Detection')

# 2. Estimated vs Realized Volatility
ax = axes[1]
ax.plot(results_df.index, results_df['vol_mean'], 'b-', linewidth=1.5, label='RBPF vol_mean')
ax.plot(results_df.index, results_df['realized_vol_20'], 'r--', linewidth=1, alpha=0.7, label='Realized (20d)')
ax.plot(results_df.index, results_df['realized_vol_exp'], 'g:', linewidth=1, alpha=0.7, label='Realized (EWM)')
ax.set_ylabel('Volatility')
ax.set_title('RBPF Volatility vs Realized')
ax.legend(loc='upper right')

# 3. Regime Probabilities (stacked)
ax = axes[2]
for r in range(N_REGIMES):
    bottom = results_df[[f'regime_prob_{i}' for i in range(r)]].sum(axis=1) if r > 0 else 0
    ax.fill_between(results_df.index, bottom, bottom + results_df[f'regime_prob_{r}'],
                    alpha=0.7, color=REGIME_COLORS[r], label=REGIME_NAMES[r])
ax.set_ylabel('Probability')
ax.set_title('Regime Probabilities')
ax.set_ylim(0, 1)
ax.legend(loc='upper right', ncol=4)

# 4. ESS and Surprise
ax = axes[3]
ax2 = ax.twinx()

ax.plot(results_df.index, results_df['ess'], 'b-', linewidth=1, label='ESS')
ax.axhline(N_PARTICLES / 2, color='blue', linestyle='--', alpha=0.5)
ax.set_ylabel('ESS', color='blue')
ax.tick_params(axis='y', labelcolor='blue')

ax2.plot(results_df.index, results_df['surprise'], 'r-', linewidth=0.8, alpha=0.7, label='Surprise')
ax2.set_ylabel('Surprise', color='red')
ax2.tick_params(axis='y', labelcolor='red')

ax.set_title('Filter Health: ESS and Surprise')

plt.tight_layout()
plt.show()

In [ ]:
# Learned parameters over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Learned μ_vol
ax = axes[0]
for r in range(N_REGIMES):
    ax.plot(results_df.index, results_df[f'learned_mu_{r}'], 
            color=REGIME_COLORS[r], linewidth=1, label=f'{REGIME_NAMES[r]}')
    # Show initial value
    ax.axhline(REGIME_PARAMS[r][1], color=REGIME_COLORS[r], linestyle='--', alpha=0.3)
ax.set_ylabel('Learned μ_vol')
ax.set_title('Storvik Parameter Learning: μ_vol per Regime')
ax.legend(loc='upper right', ncol=4)

# Outlier fraction
ax = axes[1]
ax.fill_between(results_df.index, 0, results_df['outlier_fraction'], 
                alpha=0.7, color='purple')
ax.set_ylabel('Outlier Fraction')
ax.set_title('OCSN Outlier Detection')
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 6. Metrics

In [ ]:
# Compute error metrics vs realized vol
valid = ~np.isnan(results_df['realized_vol_20'])

rbpf_vol = results_df.loc[valid, 'vol_mean'].values
rv_vol = results_df.loc[valid, 'realized_vol_20'].values

# RMSE
rmse = np.sqrt(np.mean((rbpf_vol - rv_vol) ** 2))
mae = np.mean(np.abs(rbpf_vol - rv_vol))
corr = np.corrcoef(rbpf_vol, rv_vol)[0, 1]

print("RBPF vs Realized Vol (20d):")
print(f"  RMSE: {rmse:.6f}")
print(f"  MAE:  {mae:.6f}")
print(f"  Corr: {corr:.4f}")

# Regime statistics
print(f"\nRegime Statistics:")
for r in range(N_REGIMES):
    count = (results_df['dominant_regime'] == r).sum()
    pct = count / len(results_df) * 100
    avg_vol = results_df.loc[results_df['dominant_regime'] == r, 'vol_mean'].mean()
    print(f"  {REGIME_NAMES[r]:8s}: {count:4d} ticks ({pct:5.1f}%), avg vol = {avg_vol:.4f}")

# Filter health
print(f"\nFilter Health:")
print(f"  Avg ESS: {results_df['ess'].mean():.1f} / {N_PARTICLES}")
print(f"  Min ESS: {results_df['ess'].min():.1f}")
print(f"  Resamples: {results_df['resampled'].sum()} ({results_df['resampled'].mean()*100:.1f}%)")
print(f"  Circuit breaker trips: {rbpf.circuit_breaker_trips}")

## 7. Save Results

In [ ]:
if SAVE_CSV:
    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{OUTPUT_DIR}/{TICKER}_rbpf_{PERIOD}_{timestamp}.csv"
    
    # Save
    results_df.to_csv(filename)
    print(f"Saved results to: {filename}")
    print(f"  Rows: {len(results_df)}")
    print(f"  Columns: {list(results_df.columns)}")

## 8. Interactive Tuning (Optional)

Uncomment and run to experiment with different parameters.

In [ ]:
# # Quick re-run with different parameters
# 
# def run_rbpf_quick(returns, dates, mu_calm, mu_crisis, stickiness):
#     """Quick RBPF run with simplified params"""
#     # Interpolate regime params
#     mu_range = np.linspace(mu_calm, mu_crisis, 4)
#     sigma_range = np.linspace(0.08, 0.64, 4)
#     theta_range = np.linspace(0.003, 0.12, 4)
#     
#     rbpf = RBPF(512, 4)
#     for r in range(4):
#         rbpf.set_regime_params(r, theta_range[r], mu_range[r], sigma_range[r])
#     
#     # Transition matrix with given stickiness
#     trans = np.eye(4) * stickiness
#     off_diag = (1 - stickiness) / 3
#     trans[trans == 0] = off_diag
#     rbpf.set_transition_matrix(trans.astype(np.float32))
#     
#     rbpf.enable_adaptive_forgetting(AdaptSignal.REGIME)
#     rbpf.enable_circuit_breaker(0.999, 100)
#     rbpf.enable_ocsn()
#     for r in range(4):
#         rbpf.set_ocsn_params(r, 0.02 + r * 0.01, 100 + r * 20)
#     
#     rbpf.init(mu0=mu_calm, var0=0.1)
#     
#     vols = []
#     regimes = []
#     for ret in returns:
#         rbpf.step(ret)
#         vols.append(rbpf.vol_mean)
#         regimes.append(rbpf.dominant_regime)
#     
#     return np.array(vols), np.array(regimes)
# 
# # Test different mu_calm values
# for mu_calm in [-5.0, -4.5, -4.0]:
#     vols, regimes = run_rbpf_quick(returns, dates, mu_calm, -2.0, 0.92)
#     rmse = np.sqrt(np.nanmean((vols - rv_20) ** 2))
#     print(f"mu_calm={mu_calm}: RMSE={rmse:.6f}, Crisis%={100*(regimes==3).mean():.1f}")

---

## Summary

This notebook demonstrates the full RBPF workflow:

1. **Data Fetching**: yFinance integration for any ticker
2. **Filter Configuration**: Regime params, transitions, OCSN, adaptive forgetting
3. **Execution**: Tick-by-tick processing
4. **Visualization**: Price/regime overlay, volatility comparison, regime probabilities
5. **Metrics**: RMSE vs realized vol, regime statistics, filter health
6. **Export**: CSV for further analysis

### Next Steps

- Tune regime parameters for your specific asset
- Compare RBPF volatility forecast to actual future realized vol
- Use regime detection for position sizing (Kelly)
- Extend to intraday data for HFT applications